In [1]:
import sys
import numpy as np
import pandas as pd
import torch
import pickle
import random
import math
import seaborn as sns
import torch
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import OrderedDict
from sklearn.model_selection import train_test_split

from torch import nn, optim
import torch.autograd as autograd
from torch.nn import functional as F
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

SEED = 42

def deterministic(seed):
    """
    Setup execution state so that we can reproduce multiple executions.
    Make the execution "as deterministic" as possible.

    random_seed: seed used to feed torch, numpy and python random
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed_all(seed)      
       
    
deterministic(SEED)
    
    
    
class _ECELoss(nn.Module):
    """
    Calculates the Expected Calibration Error of a model.
    (This isn't necessary for temperature scaling, just a cool metric).
    The input to this loss is the logits of a model, NOT the softmax scores.
    This divides the confidence outputs into equally-sized interval bins.
    In each bin, we compute the confidence gap:
    bin_gap = | avg_confidence_in_bin - accuracy_in_bin |
    We then return a weighted average of the gaps, based on the number
    of samples in each bin
    See: Naeini, Mahdi Pakdaman, Gregory F. Cooper, and Milos Hauskrecht.
    "Obtaining Well Calibrated Probabilities Using Bayesian Binning." AAAI.
    2015.
    """
    def __init__(self, n_bins=15):
        """
        n_bins (int): number of confidence interval bins
        """
        super(_ECELoss, self).__init__()
        bin_boundaries = torch.linspace(0, 1, n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]

    def forward(self, logits, labels):
        softmaxes = F.softmax(logits, dim=1)
        confidences, predictions = torch.max(softmaxes, 1)
        accuracies = predictions.eq(labels)

        ece = torch.zeros(1, device=logits.device)
        for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
            prop_in_bin = in_bin.float().mean()
            if prop_in_bin.item() > 0:
                accuracy_in_bin = accuracies[in_bin].float().mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

        return ece
       
def get_nll_ece(logits, logits_scaled, labels):
    nll_criterion = nn.CrossEntropyLoss().cuda()
    ece_criterion = _ECELoss(n_bins=10).cuda()
    
    nll0 = nll_criterion(logits, labels).item()
    ece0 = ece_criterion(logits, labels).item()

    nll1 = nll_criterion(logits_scaled, labels).item()
    ece1 = ece_criterion(logits_scaled, labels).item()

    print(f'+-----+---------+---------+')
    print(f'|     | Before  |  After  |')
    print(f'+-----+---------+---------+')
    print(f'| NLL | {nll0:7.5f} | {nll1:6.5f} |')
    print(f'| ECE | {ece0:7.5f} | {ece1:6.5f} |')
    print(f'+-----+---------+---------+')

    return (nll0, ece0), (nll1, ece1)    
    
def get_logits_labels(model, data_loader):
    logits_list = []
    labels_list = []
    model.eval()
    with torch.no_grad():
        for input, label in data_loader:
            input = input.cuda()
            logits = model(input)
            logits_list.append(logits)
            labels_list.append(label.long())
        logits = torch.cat(logits_list).cuda()
        labels = torch.cat(labels_list)

    labels = torch.cuda.LongTensor([label for idx, label in np.argwhere(labels.numpy())])
    
    return logits, labels  

class ModelWithTemperature(nn.Module):
    """
    A thin decorator, which wraps a model with temperature scaling
    model (nn.Module):
        A classification neural network
        NB: Output of the neural network should be the classification logits,
            NOT the softmax (or log softmax)!
    """
    def __init__(self, model, initial_temperature=1.5):
        super(ModelWithTemperature, self).__init__()
        self.model = model
        self.temperature = nn.Parameter(torch.ones(1) * initial_temperature)
        self.cuda()
        self.nll_criterion = nn.CrossEntropyLoss().cuda()
        
    def forward(self, input):
        logits = self.model(input)
        return self.temperature_scale(logits)

    def temperature_scale(self, logits):
        """
        Perform temperature scaling on logits
        """
        # Expand temperature to match the size of logits
        temperature = self.temperature.unsqueeze(1).expand(logits.size(0), logits.size(1))
        return logits / temperature            
    
    # This function probably should live outside of this class, but whatever
    def set_temperature(self, logits, labels):
        """
        Tune the tempearature of the model (using the validation set).
        We're going to set it to optimize NLL.
        data_loader (DataLoader): validation set loader
        """                
        optimizer = optim.LBFGS([self.temperature], lr=0.01, max_iter=50)
        
        def eval():
            loss = self.nll_criterion(self.temperature_scale(logits), labels)
            loss.backward()
            return loss
        optimizer.step(eval)
        
        print('Optimal temperature = %.3f' % self.temperature.item())
        
class ATS(nn.Module):

    def __init__(self, initial_temperature=1.5):
        super().__init__()
        self.temperature = autograd.Variable(torch.ones(1)*initial_temperature, requires_grad=True)
        
    def attended_NLL(self,logits, considered_labels, true_labels):
        soft_logit = F.softmax(logits , dim=1).cuda()
        loss = 0
        for i in range(soft_logit.shape[0]):
            if (true_labels[i].item() == considered_labels[i].item()):
                loss = loss - 0.001*torch.log((((soft_logit[i,considered_labels[i]]).cuda()+(1e-8))/(1-(soft_logit[i,considered_labels[i]]).cuda()+(1e-8)))*(1-soft_logit[i,true_labels[i]].cuda()+(1e-8)))
            else:
                loss = loss - 0.001*torch.log((((soft_logit[i,considered_labels[i]]).cuda()+(1e-8))/(1-(soft_logit[i,considered_labels[i]]).cuda()+(1e-8)))*(soft_logit[i,considered_labels[i]].cuda()+(1e-8)))
        return loss

    def select_approperate_samples_for_ATS(self, logits, labels, threshold):
        considered_class = []
        considered_class_labels = []
        true_labels = []
        confidence = F.softmax(logits,dim=1)

        for i in range(torch.max(labels)+1):

            considered_class.append(logits[(labels == i),:])        
            considered_class_labels.append(labels[(labels == i)])
            true_labels.append(labels[(labels == i)])

            b = torch.mul((logits.max(1)[1]==i),(labels!=i))
            

            considered_class.append(logits[(b==1),:])
            lab = torch.ones(labels[(b==1)].size()[0], dtype=torch.long)*i
            lab = lab.cuda()
            considered_class_labels.append(lab) 
            true_labels.append(labels[(b==1)]) 

            b = torch.mul((logits.max(1)[1]!=i),(labels!=i))
            c = torch.mul(b,(confidence[:,i]>threshold))
            considered_class.append(logits[(c==1),:])
            lab = torch.ones(labels[(c==1)].size()[0], dtype=torch.long)*i

            lab = lab.cuda()

            considered_class_labels.append(lab) 
            true_labels.append(labels[(c==1)]) 

        considered_class = torch.cat(considered_class)
        considered_class_labels = torch.cat(considered_class_labels)    
        true_labels = torch.cat(true_labels)

        return considered_class,considered_class_labels, true_labels

    def find_best_T(self, logits, labels, threshold):
        considered_class,considered_labels, true_labels = self.select_approperate_samples_for_ATS(logits, labels, threshold)
        logits = logits.cuda()
        considered_class = considered_class.cuda()
        optimizer = optim.LBFGS([self.temperature], lr=0.01, max_iter=50)

        def eval():
            L = considered_class / self.temperature.unsqueeze(1).cuda()
            loss = self.attended_NLL(L,considered_labels, true_labels).cuda()			
            loss = loss.cuda()
            loss.backward(retain_graph=True )
            return loss
        
        optimizer.step(eval)
        return self.temperature
    
def split_eval_dataset(model, dataset, validation_size=0.1, validation_labels_noise=0):
    """
    Evaluates the whole dataset using the given model and splits the logits and labels into two datasets
    using the validation_size proportion. If you want to add noise on the validation set labels, use the 
    parameter validation_labels_noise with range [0,1].
    """
    idxs = range(len(dataset))
    idxsvalid, idxstest = train_test_split(idxs, test_size=(1-validation_size), random_state=42, stratify=dataset.labels)
    dsvalid = Subset(dataset, idxsvalid)
    valid_loader = DataLoader(dsvalid, 128, num_workers=1, pin_memory=True)
    dstest = Subset(dataset, idxstest)
    test_loader = DataLoader(dstest, 128, num_workers=1, pin_memory=True)
    
    logits_valid, labels_valid = get_logits_labels(model, valid_loader)
    logits_test, labels_test = get_logits_labels(model, test_loader)
    
    # Apply noise randomly
    if validation_labels_noise > 0:
        num_classes = logits_valid.shape[-1]-1
        idxs_valid = list(range(len(labels_valid)))
        random.shuffle(idxs_valid)
        for idx in idxs_valid[:int(validation_labels_noise*len(idxs_valid))]:
            labels_valid[idx] = random.randint(0,num_classes)
        #labels_valid = torch.cuda.LongTensor([random.randint(0,num_classes) if random.random() < validation_labels_noise else label for label in labels_valid])
    
    return logits_valid, labels_valid, logits_test, labels_test


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

class CIFAR100(Dataset):
    """
    Download CIFAR100 from https://www.cs.toronto.edu/~kriz/cifar.html.
    After extraction, reference the dataset root using the parameter base_dir.    
    """
    
    def __init__(self, base_dir="./data", dataset='train', offset=0, limit=None, verbose=True):
        self.set = unpickle(f'{base_dir}/cifar100/{dataset}')
        self.meta = unpickle(f'{base_dir}/cifar100/meta')
        self.fine_label_names = [t.decode('utf8') for t in self.meta[b'fine_label_names']]
        self.data = self.set[b'data']

        self.labels = np.zeros((len(self.data), 100))
        for idx, label in enumerate(self.set[b'fine_labels']):
            self.labels[idx][label] = 1

        self.offset = offset if offset > 0 else 0
        self.limit = len(self.data) if limit is None else limit

        self.data = self.data[offset:limit]
        self.labels = self.labels[offset:limit]
        print(self.labels.shape)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        d = self.data[idx]
        image = np.zeros((32, 32, 3), dtype=np.uint8)
        image[..., 0] = np.reshape(d[:1024], (32, 32))  # Red channel
        image[..., 1] = np.reshape(d[1024:2048], (32, 32))  # Green channel
        image[..., 2] = np.reshape(d[2048:], (32, 32))  # Blue channel

        return transforms.ToTensor()(image), self.labels[idx]

In [3]:
"""
DenseNet for cifar with pytorch

Reference:
[1] H. Gao, Z. Liu, L. Maaten and K. Weinberger. Densely connected convolutional networks. In CVPR, 2017
"""

class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                        growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                        kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))


class DenseNet_Cifar(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_

    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
    """
    def __init__(self, growth_rate=12, block_config=(16, 16, 16),
                 num_init_features=24, bn_size=4, drop_rate=0, num_classes=10):

        super(DenseNet_Cifar, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=3, stride=1, padding=1, bias=False)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)
        
        # initialize conv and bn parameters
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.avg_pool2d(out, kernel_size=8, stride=1).view(features.size(0), -1)
        out = self.classifier(out)
        return out


def densenet_BC_cifar(depth, k, **kwargs):
    N = (depth - 4) // 6
    model = DenseNet_Cifar(growth_rate=k, block_config=[N, N, N], num_init_features=2*k, **kwargs)
    return model

In [4]:

dataset = CIFAR100(base_dir='./data', dataset='test', verbose=True)

model = DenseNet_Cifar(block_config=(6, 6, 6), num_classes=100)





model_state = torch.load('./model/model_70.pth')
# model_state = OrderedDict([(key.replace("module.",""), value) for key, value in model_state['state_dict'].items()])
model.load_state_dict(model_state)
model.cuda()

(10000, 100)


DenseNet_Cifar(
  (features): Sequential(
    (conv0): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(24, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(36, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (re

### Comparing TS with ATS exploring different thresholds with Bayesian Optimization

In [5]:
 # Noise = 0

In [50]:
initial_temperature = 3
split_size = 0.02

logits_valid, labels_valid, logits_test, labels_test = split_eval_dataset(model, dataset, split_size)

In [51]:
# TS
model_temp = ModelWithTemperature(model, initial_temperature=initial_temperature) 
model_temp.set_temperature(logits_valid, labels_valid)

logits_valid_scaled_ts = model_temp.temperature_scale(logits_valid)      
# logits_test_scaled_ts = model_temp.temperature_scale(logits_test)     

print(f"# validation_size = {len(logits_valid)}")
get_nll_ece(logits_valid, logits_valid_scaled_ts, labels_valid)    
print(f"# test_size = {len(logits_test)}")
get_nll_ece(logits_test, logits_test_scaled_ts, labels_test) 

Optimal temperature = 2.931
# validation_size = 200
+-----+---------+---------+
|     | Before  |  After  |
+-----+---------+---------+
| NLL | 5.33487 | 3.69636 |
| ECE | 0.36361 | 0.03984 |
+-----+---------+---------+


((5.334865093231201, 0.3636055290699005),
 (3.696359872817993, 0.03984362259507179))

In [53]:
%%time
# ATS
model_ATS = ATS(initial_temperature=initial_temperature)
T_ATS = model_ATS.find_best_T(logits_valid, labels_valid, threshold=1e-5)
print("Optimal temperature: ",T_ATS.item())

logits_valid_scaled_ats = logits_valid/T_ATS.cuda()
logits_test_scaled_ats = logits_test/T_ATS.cuda()

print(f"# validation_size = {len(logits_valid)}")
get_nll_ece(logits_valid, logits_valid_scaled_ats, labels_valid)
print(f"# test_size = {len(logits_test)}")
get_nll_ece(logits_test, logits_test_scaled_ats, labels_test)  

Optimal temperature:  2.3912477493286133
# validation_size = 200
+-----+---------+---------+
|     | Before  |  After  |
+-----+---------+---------+
| NLL | 5.33487 | 3.73758 |
| ECE | 0.36361 | 0.06200 |
+-----+---------+---------+
# test_size = 9800
+-----+---------+---------+
|     | Before  |  After  |
+-----+---------+---------+
| NLL | 5.17957 | 3.63603 |
| ECE | 0.37831 | 0.05170 |
+-----+---------+---------+
CPU times: user 34.6 s, sys: 3.98 ms, total: 34.6 s
Wall time: 34.6 s


((5.179574966430664, 0.3783113360404968),
 (3.6360340118408203, 0.05169958621263504))

In [17]:
 # Noise = 0.05

In [63]:
initial_temperature = 3
split_size = 0.02

logits_valid, labels_valid, logits_test, labels_test = split_eval_dataset(model, dataset, split_size,validation_labels_noise=0.05)

In [64]:
# TS
model_temp = ModelWithTemperature(model, initial_temperature=initial_temperature) 
model_temp.set_temperature(logits_valid, labels_valid)

logits_valid_scaled_ts = model_temp.temperature_scale(logits_valid)      
logits_test_scaled_ts = model_temp.temperature_scale(logits_test)     

print(f"# validation_size = {len(logits_valid)}")
get_nll_ece(logits_valid, logits_valid_scaled_ts, labels_valid)    
print(f"# test_size = {len(logits_test)}")
get_nll_ece(logits_test, logits_test_scaled_ts, labels_test) 

Optimal temperature = 4.000
# validation_size = 200
+-----+---------+---------+
|     | Before  |  After  |
+-----+---------+---------+
| NLL | 5.50194 | 3.79190 |
| ECE | 0.36361 | 0.06130 |
+-----+---------+---------+
# test_size = 9800
+-----+---------+---------+
|     | Before  |  After  |
+-----+---------+---------+
| NLL | 5.17957 | 3.67798 |
| ECE | 0.37831 | 0.07266 |
+-----+---------+---------+


((5.179574966430664, 0.3783113360404968),
 (3.67798113822937, 0.07265952974557877))

In [65]:
%%time
# ATS
model_ATS = ATS(initial_temperature=initial_temperature)
T_ATS = model_ATS.find_best_T(logits_valid, labels_valid, threshold=1e-5)
print("Optimal temperature: ",T_ATS.item())
logits_valid_scaled_ats = logits_valid/T_ATS.cuda()
logits_test_scaled_ats = logits_test/T_ATS.cuda()

print(f"# validation_size = {len(logits_valid)}")
get_nll_ece(logits_valid, logits_valid_scaled_ats, labels_valid)
print(f"# test_size = {len(logits_test)}")
get_nll_ece(logits_test, logits_test_scaled_ats, labels_test)  

Optimal temperature:  2.180772542953491
# validation_size = 200
+-----+---------+---------+
|     | Before  |  After  |
+-----+---------+---------+
| NLL | 5.50194 | 3.86166 |
| ECE | 0.36361 | 0.08626 |
+-----+---------+---------+
# test_size = 9800
+-----+---------+---------+
|     | Before  |  After  |
+-----+---------+---------+
| NLL | 5.17957 | 3.67766 |
| ECE | 0.37831 | 0.07607 |
+-----+---------+---------+
CPU times: user 53.7 s, sys: 91.9 ms, total: 53.8 s
Wall time: 53.8 s


((5.179574966430664, 0.3783113360404968),
 (3.6776621341705322, 0.07607250660657883))